# 1d Euler Equation

**Goal:** Solve the 1d Euler's equation using a finite volume scheme.

The equation reads $\partial_t \mathbf{u} + \partial_x \mathbf{F} = 0$ with
$$
\mathbf{u} = \begin{pmatrix} \rho \\ \rho v_x \\ E \end{pmatrix} \; ,
$$
and the flux
$$
\mathbf{F} = \begin{pmatrix} \rho v_x \\ \rho v_x^2 + p \\ (E + p) v_x \end{pmatrix} \; ,
$$
where we defined the total energy $E = \rho e + \frac{1}{2} \rho v_x^2$ with $e$ the internal energy.

We use the equation of state of a polytropic ideal gas $p(e,\rho)=\rho e (\gamma - 1)$ with $\gamma = \frac{5}{3}$. 

In [1]:
using DifferentialEquations
using GLMakie

In [2]:
struct Grid
    xmin::Float64
    xmax::Float64
    Nx::Int64
    xl::Vector{Float64}
    Δx::Float64

    function Grid(; xmin=0, xmax=1, Nx=100)
        xl = range(xmin, xmax, Nx)
        Δx = xl[2] - xl[1]
        new(xmin, xmax, Nx, xl, Δx)
    end
end

In [15]:
function get_reconstruct(gd::Grid, ρl, vl, pl)
    wboundaries = (ρ=zeros(gd.Nx, 2), v=zeros(gd.Nx, 2), p=zeros(gd.Nx, 2))

    for i in 1:gd.Nx
        im = 1 == 1 ? gd.Nx : i - 1
        ip = i == gd.Nx ? 1 : i + 1
        ipp = i == gd.Nx ? 2 : i == gd.Nx - 1 ? 1 : i + 2

        κ = 1 / 3

        # wboundaries.ρ[i, 1] = ρl[i]
        # wboundaries.ρ[i, 2] = ρl[ip]
        # wboundaries.v[i, 1] = vl[i]
        # wboundaries.v[i, 2] = vl[ip]
        # wboundaries.p[i, 1] = pl[i]
        # wboundaries.p[i, 2] = pl[ip]

        wboundaries.ρ[i, 1] = ρl[i] + 1 / 4 * ((1 - κ) * (ρl[i] - ρl[im]) + (1 + κ) * (ρl[ip] - ρl[i]))
        wboundaries.ρ[i, 2] = ρl[ip] - 1 / 4 * ((1 + κ) * (ρl[ip] - ρl[i] + (1 - κ) * (ρl[ipp] - ρl[ip])))
        wboundaries.v[i, 1] = vl[i] + 1 / 4 * ((1 - κ) * (vl[i] - vl[im]) + (1 + κ) * (vl[ip] - vl[i]))
        wboundaries.v[i, 2] = vl[ip] - 1 / 4 * ((1 + κ) * (vl[ip] - vl[i] + (1 - κ) * (vl[ipp] - vl[ip])))
        wboundaries.p[i, 1] = pl[i] + 1 / 4 * ((1 - κ) * (pl[i] - pl[im]) + (1 + κ) * (pl[ip] - pl[i]))
        wboundaries.p[i, 2] = pl[ip] - 1 / 4 * ((1 + κ) * (pl[ip] - pl[i] + (1 - κ) * (pl[ipp] - pl[ip])))
    end

    return wboundaries
end

function flux(ρ, v, p)
    e = p / ρ * (5 / 3 - 1)

    return [
        ρ * v,
        ρ * v^2 + p,
        (ρ * e + 1 / 2 * ρ * v^2 + p) * v
    ]
end

function get_flux(gd::Grid, ρl, vl, pl)
    Fl = zeros(gd.Nx, 3)

    for i in 1:length(ρl)
        wbound = get_reconstruct(gd, ρl, vl, pl)

        Fminus = flux(wbound.ρ[i, 1], wbound.v[i, 1], wbound.p[i, 1])

        Fplus = flux(wbound.ρ[i, 2], wbound.v[i, 2], wbound.p[i, 2])

        @. Fl[i, :] = 1 / 2 * (Fplus + Fminus)
        # @. Fl[i, :] = Fplus
        # @. Fl[i, :] = Fminus
    end

    return Fl
end

function euler!(du, u, p, t)
    (; gd) = p

    F = get_flux(gd, u[1, :], u[2, :], u[3, :])

    for i in 1:gd.Nx
        im = i == 1 ? gd.Nx : i - 1
        @. du[:, i] = -(F[i, :] - F[im, :])
    end
end

euler! (generic function with 1 method)

In [16]:
gd = Grid(xmin=0, xmax=1, Nx=100)

# ρ0l = ones(gd.Nx)
ρ0l = map(x -> exp(-100 * (x - 0.5)^2), gd.xl)
# v0l = zeros(gd.Nx)
v0l = map(x -> exp(-100 * (x - 0.5)^2), gd.xl)
p0l = ones(gd.Nx)
# p0l = map(x -> exp(-100 * (x - 0.5)^2), gd.xl)

get_reconstruct(gd, ρ0l, v0l, p0l)
get_flux(gd, ρ0l, v0l, p0l);

In [19]:
u0 = zeros(3, gd.Nx)
u0[1, :] .= ρ0l
u0[2, :] .= v0l
u0[3, :] .= p0l

tspan = (0, 20)

prob = ODEProblem(euler!, u0, tspan, (; gd))
sol = solve(prob)

retcode: Success
Interpolation: specialized 4th order "free" interpolation, 3rd order Hermite
t: 38-element Vector{Float64}:
  0.0
  0.15387257549287453
  0.4225577197002945
  0.7402281332027445
  1.1276638199467488
  1.5574892069513533
  2.0253103608616887
  2.515618459519829
  3.0232687553619075
  3.5445362357872203
  ⋮
 15.436603195899803
 16.06357280581299
 16.690524162941724
 17.319194570694
 17.95029070063016
 18.58181198827872
 19.21452387844308
 19.844770686456368
 20.0
u: 38-element Vector{Matrix{Float64}}:
 [1.3887943864964021e-11 3.774749385421681e-11 … 3.774749385421654e-11 1.3887943864964021e-11; 1.3887943864964021e-11 3.774749385421681e-11 … 3.774749385421654e-11 1.3887943864964021e-11; 1.0 1.0 … 1.0 1.0]
 [1.3887943864928378e-11 3.7747493853822316e-11 … 3.77474938550964e-11 1.3887943865069147e-11; 1.436725654589907e-11 3.841933590139815e-11 … 3.9026378497827526e-11 1.4522030118728748e-11; 0.9999999999980559 0.9999999999911676 … 1.000000000013258 1.0000000000042912]
 [1.3

In [20]:
fig = Figure(size=(1200, 400))

sg = SliderGrid(fig[2, 1], (label="t", range=range(tspan[1], tspan[2], 100)))
tlift = sg.sliders[1].value

ax1 = Axis(fig[1, 1])
lines!(ax1, gd.xl, u0[1, :])
lines!(ax1, gd.xl, (@lift sol($tlift)[1, :]))

ax2 = Axis(fig[1, 2])
lines!(ax2, gd.xl, u0[2, :])
lines!(ax2, gd.xl, (@lift sol($tlift)[2, :]))

ax3 = Axis(fig[1, 3])
lines!(ax3, gd.xl, u0[3, :])
lines!(ax3, gd.xl, (@lift sol($tlift)[3, :]))

fig